# <center><span style = " font-size:2em;">3조_임재원_스팸문자분류</span></center>
---
# 목차
- 데이터 로딩 및 확인
- 데이터 전처리 및 병합
- 문자내용 전처리
- 학습 / 테스트 데이터 분리
- 모델 생성 및 확인
- 외부데이터로 모델 검증
- 아쉬웠던 점  

# 데이터
- SPAM 문자 데이터 : https://www.bigdata-policing.kr/product/view?product_id=PRDT_395(인피니그루 스팸 문자 데이터)
- 정상 문자 데이터1 : https://www.data.go.kr/data/15077116/fileData.do(서울시 구청 행정문자 발송현황)
- 정상 문자 데이터2 : https://www.data.go.kr/data/15089190/fileData.do(환경산업기술원 챗봇 대화세트)

In [2]:
# 모듈 로딩
import numpy as np
import pandas as pd
import re
import warnings
from konlpy.tag import Kkma, Komoran, Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')

# [1] 데이터 로딩 및 확인

In [3]:
spam_1 = pd.read_csv(r'./data/spam/스팸문자내역1.csv', encoding='utf-8')
spam_2 = pd.read_csv(r'./data/spam/스팸문자내역2.csv', encoding='utf-8')
ham_1 = pd.read_csv(r'./data/ham/서울시 구청 재난문자 발송현황.csv', encoding='cp949')
ham_2 = pd.read_csv(r'./data/ham/환경산업기술원 챗봇 대화세트 구성.csv', encoding='cp949')

In [4]:
spam_1.head(5)

,index,text
0,1,[국제발신]ifg@? 명절 *만 이벤진행ifg@ifg@? 최고의 메리트ifg@ifg...
1,2,[국외발신]ifg@●빅토리●ifg@ifg@퇴근시간**분전ifg@뒷면이보일정도로ifg...
2,3,[국제발신]ifg@오빠안녕하세요ifg@관리드렸던지수에요ifg@이번에도ifg@확실하게...
3,4,[국제발신]ifg@ifg@VERTifg@ifg@처음 **%ifg@삼+삼ifg@**+...
4,5,[국외발신]ifg@?그리스?ifg@이붼츄가넘쳐유~ifg@커피/치킨???ifg@항상감...


In [5]:
ham_1.head(5)

,연,월,일,시,분,지역,송출내용
0,2021,2,1,9,46,강북구,"[강북구청]확진자5명(620~624번)발생, 동선( www.gangbuk.go.kr..."
1,2021,2,1,10,0,노원구,[노원구청] 오늘(2.1.) 오전 확진자 2명 발생(1047~1048번). 역학조사...
2,2021,2,1,10,0,광진구,[광진구청] 타지역(중랑구)확진자 관내 동선 알림.상호 공개된 업소 홈페이지( ha...
3,2021,2,1,10,9,도봉구,[도봉구청] 2.1(월) 확진자 1명 발생(748번). 역학조사결과 이동동선 및 조...
4,2021,2,1,10,13,성북구,[성북구청] 1020~1023번(4명) 확진자 발생. 거주지 방역 완료. 자세한 사...


# [2] 데이터 전처리 및 병합
- text, target의 열을 가진 새로운 데이터프레임 생성
- 불러온 데이터프레임에서 문자내역을 text열에 담고, target에 spam일 경우 1 / ham일 경우 0 지정
### But, 스팸문자가 약 17000개, 정상문자가 2425개로 데이터의 불균형이 심각함
- 스팸문자의 경우 비슷비슷한 경우가 많기때문에 스팸문자의 비율을 언더샘플링하여 정상문자와 비슷한 수로 맞춰줌

In [6]:
# 새로운 데이터프레임 생성
text_data = pd.DataFrame(columns=[['text', 'target']])

# spam, ham 문자메세지 target 생성
spam_1['target'] = 1
spam_2['target'] = 1
ham_1['target'] = 0
ham_2['target'] = 0

# concat준비
spam_1.drop(['index'], axis=1, inplace=True)
spam_2.drop(['index'], axis=1, inplace=True)
ham_1.drop(ham_1.columns[0:6], axis=1, inplace=True)
ham_2.drop(ham_2.columns[0:4], axis=1, inplace=True)
ham_2.drop(ham_2.columns[1:7], axis=1, inplace=True)
ham_1.rename(columns={'송출내용': 'text'}, inplace=True)
ham_2.rename(columns={'답변내용(4,000자 이내) - 필수': 'text'}, inplace=True)
spam_1 = spam_1.sample(n=1200, random_state=42)
spam_2 = spam_2.sample(n=1200, random_state=42)

# text_data에 병합
text_data = pd.concat([spam_1, spam_2, ham_1, ham_2], ignore_index=True)

In [7]:
# 병합된 데이터 확인
text_data

,text,target
0,[국제발신]ifg@화요일지원금ifg@도착하였습니다ifg@코::드 - ***ifg@서...,1
1,[국제발신]ifg@체리#**★타ifg@추억의당구장ifg@원바#투바#올과일ifg@**...,1
2,"[국외발신]ifg@새해엔좋은일만가득하세요ifg@적립 ***,***원ifg@ifg@l...",1
3,[국외발신]ifg@카】강원ifg@ⓖ】마카오보다ifg@노】여기로오세요ifg@올구조대'...,1
4,[국제발신]ifg@심심할때오세요ifg@시간잘~갑니다ifg@**.***원도줍니다ifg...,1
...,...,...
4820,제품사후관리실에서는 환경표지등의 인증을 받지 아니하고 환경표지 등 또는 이와 유사한...,0
4821,"가능합니다. 다만, 기 인증 제품과 환경성 및 품질에 변동사항이 없다면(원료사용 내...",0
4822,환경표지 홈페이지(http://el.keiti.re.kr) 홈 → 정보마당 → 서식...,0
4823,"파생제품 등록의 경우 현장심사나 시험의뢰를 진행하지 않으며, 서류심사를 통해 진행됩니다.",0


In [8]:
# value_counts()로 스팸, 정상문자의 수 확인
text_data['target'].value_counts()

0    2425
1    2400
Name: target, dtype: int64

# [3] 문자내용 전처리
- 영어와 특수문자 제거
- konlpy를 통한 형태소 분리(명사만 추출)
- Vectorizer를 통한 단어간 거리 수치화(문자를 계산가능한 정량적인 숫자로 바꿔주는 작업)

In [9]:
# 영어와 특수문자 제거
text_data['text'] = [re.sub('[^0-9가-힣]', '', s) for s in text_data['text']]

In [10]:
# konlpy를 통한 형태소 분리(명사만 추출)
x_data = text_data['text'].values
y_data = text_data['target'].values

for i, document in enumerate(x_data):
    words = Okt().pos(document)
    clean_words = []
    for word, tag in words:
        if tag in ['Noun']:
            if len(word) > 1:
                clean_words.append(word)
    document = ' '.join(clean_words)
    x_data[i] = document

In [11]:
# Vectorizer를 통한 단어간 거리 수치화
vectorizer = TfidfVectorizer()
vectorizer.fit(x_data)
x_data = vectorizer.transform(x_data)

In [12]:
# target 라벨 인코딩
le = LabelEncoder()
le.fit(y_data)
labels = le.classes_
y_data = le.transform(y_data)

# [4] 학습 / 테스트 데이터 분리

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42, stratify=y_data)

# [5] 모델 생성 및 확인

In [14]:
# 모델 생성(MultinomialNB)
model_1 = MultinomialNB()
model_1.fit(x_train, y_train)

# 모델 생성(LogisticRegression)
model_2 = LogisticRegression()
model_2.fit(x_train, y_train)

LogisticRegression()

In [15]:
# Gridserach로 MultinomialNB 최적의 파라미터 확인, 교차검증 5회 실시
paramsNB = {'alpha':[0.1, 0.5, 1, 5, 10]}

grid = GridSearchCV(model_1, param_grid=paramsNB, scoring='accuracy', cv=5)
grid.fit(x_train, y_train)
print(f'교차검증 score : {grid.best_score_}')
print(f'LogisticRegression 최적의 파라미터 모델 : {grid.best_estimator_}')

교차검증 score : 0.9979268025421872
LogisticRegression 최적의 파라미터 모델 : MultinomialNB(alpha=0.1)


In [16]:
# Gridserach로 LogisticRegression 최적의 파라미터 확인, 교차검증 5회 실시
paramsLR = {'penalty':['l2', 'l1'],
         'C':[0.01, 0.1, 1, 5, 10]}

grid = GridSearchCV(model_2, param_grid=paramsLR, scoring='accuracy', cv=5)
grid.fit(x_train, y_train)
print(f'교차검증 score : {grid.best_score_}')
print(f'LogisticRegression 최적의 파라미터 모델 : {grid.best_estimator_}')

교차검증 score : 0.9991111111111112
LogisticRegression 최적의 파라미터 모델 : LogisticRegression(C=5)


In [17]:
# 도출된 하이퍼 파라미터를 바탕으로 최적의 모델 생성
model_1 = MultinomialNB(alpha=0.1)
model_1.fit(x_train, y_train)

model_2 = LogisticRegression(C=5)
model_2.fit(x_train, y_train)

LogisticRegression(C=5)

In [18]:
# 모델 스코어 확인
print(f'MultinomialNB train score : {model_1.score(x_train, y_train)}')
print(f'MultinomialNB test score : {model_1.score(x_test, y_test)}')

print(f'LogisticRegression train score : {model_2.score(x_train, y_train)}')
print(f'LogisticRegression train score : {model_2.score(x_train, y_train)}')

MultinomialNB train score : 1.0
MultinomialNB test score : 0.9979281767955801
LogisticRegression train score : 0.9994077583654131
LogisticRegression train score : 0.9994077583654131


# [6] 외부 데이터로 모델 검증

In [19]:
# 검증 함수 생성
def predict_multinomial(temp_text):
    global words, clean_words, word, tag
    text = temp_text
    words = Okt().pos(temp_text)
    clean_words = []

    for word, tag in words:
        if tag in ['Noun']:
            if len(word) > 1:
                clean_words.append(word)
    document = ' '.join(clean_words)
    print(f'문자 내용 : {text}')
    print(f'===========================================================')

    x_test_temp = [document]
    x_test_temp = vectorizer.transform(x_test_temp)

    y_predict = model_1.predict(x_test_temp)
    y_proba = model_1.predict_proba(x_test_temp)
    print('MultinomialNB model 예측결과')
    if labels[y_predict[0]] == 0:
        print(f'정상 문자 입니다. / 스팸 문자일 확률 : {np.round(y_proba[0][-1], 4) * 100}%')
    if labels[y_predict[0]] == 1:
        print(f'스팸 문자 입니다. / 스팸 문자일 확률 : {np.round(y_proba[0][-1], 4) * 100}%')

def predict_logistic(temp_text):
    global words, clean_words, word, tag
    words = Okt().pos(temp_text)
    clean_words = []

    for word, tag in words:
        if tag in ['Noun']:
            if len(word) > 1:
                clean_words.append(word)
    document = ' '.join(clean_words)
    print('LogisticRegression model 예측결과')

    x_test_temp = [document]
    x_test_temp = vectorizer.transform(x_test_temp)

    y_predict_2 = model_2.predict(x_test_temp)
    y_proba_2 = model_2.predict_proba(x_test_temp)
    if labels[y_predict_2[0]] == 0:
        print(f'정상 문자 입니다. / 스팸 문자일 확률 : {np.round(y_proba_2[0][-1], 4) * 100}%')
    if labels[y_predict_2[0]] == 1:
        print(f'스팸 문자 입니다. / 스팸 문자일 확률 : {np.round(y_proba_2[0][-1], 4) * 100}%')

# 실제로 온 스팸문자 3개 테스트

In [20]:
predict_multinomial('J컴퍼니 인터넷+(IP)TV 신규&교체 580,000 무료거부 080')
predict_logistic('J컴퍼니 인터넷+(IP)TV 신규&교체 580,000 무료거부 080')

문자 내용 : J컴퍼니 인터넷+(IP)TV 신규&교체 580,000 무료거부 080
MultinomialNB model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 90.73%
LogisticRegression model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 73.03%


In [22]:
predict_multinomial('※여름 바다여행※ 현금 50,000으로 호캉스 즐기자 코드 999필수')
predict_logistic('※여름 바다여행※ 현금 50,000으로 호캉스 즐기자 코드 999필수')

문자 내용 : ※여름 바다여행※ 현금 50,000으로 호캉스 즐기자 코드 999필수
MultinomialNB model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 96.6%
LogisticRegression model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 74.67%


In [23]:
predict_multinomial('★루이★ 카진오 신규모~집 여러가지 혜택 활동.매일.월별 1:1 체계관리')
predict_logistic('★루이★ 카진오 신규모~집 여러가지 혜택 활동.매일.월별 1:1 체계관리')

문자 내용 : ★루이★ 카진오 신규모~집 여러가지 혜택 활동.매일.월별 1:1 체계관리
MultinomialNB model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 89.5%
LogisticRegression model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 55.96%


# 정상 문자 3개 테스트

In [24]:
predict_multinomial('[대구광역시] 9월 8일 코로나 확진자 3931명(달서구 978, 북구 678, 수성구 558)')
predict_logistic('[대구광역시] 9월 8일 코로나 확진자 3931명(달서구 978, 북구 678, 수성구 558)')

문자 내용 : [대구광역시] 9월 8일 코로나 확진자 3931명(달서구 978, 북구 678, 수성구 558)
MultinomialNB model 예측결과
정상 문자 입니다. / 스팸 문자일 확률 : 0.02%
LogisticRegression model 예측결과
정상 문자 입니다. / 스팸 문자일 확률 : 0.27%


In [26]:
predict_multinomial('날이 참 좋네요, 경북대학교에 사람이 많습니다.')
predict_logistic('날이 참 좋네요, 경북대학교에 사람이 많습니다.')

문자 내용 : 날이 참 좋네요, 경북대학교에 사람이 많습니다.
MultinomialNB model 예측결과
정상 문자 입니다. / 스팸 문자일 확률 : 13.3%
LogisticRegression model 예측결과
정상 문자 입니다. / 스팸 문자일 확률 : 43.85%


In [27]:
predict_multinomial('[CJ대한통운 택배 배송 완료] 고객님의 상품이 배송 완료 되었습니다.')
predict_logistic('[CJ대한통운 택배 배송 완료] 고객님의 상품이 배송 완료 되었습니다.')

문자 내용 : [CJ대한통운 택배 배송 완료] 고객님의 상품이 배송 완료 되었습니다.
MultinomialNB model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 97.99%
LogisticRegression model 예측결과
스팸 문자 입니다. / 스팸 문자일 확률 : 52.88%


# [7] 아쉬웠던 점
- 스팸문자보다 정상문자의 데이터가 너무 작아서 스팸문자를 언더샘플링한 점
- 정상문자를 다양하게 구하지 못해서 재난경보, 챗봇 답변만을 쓸수밖에 없었던 점 -> 그래서 일상 문자의 예측률이 떨어진다